In [1]:
from ImagePreprocessing import *
import time

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [2]:
#df = pd.read_csv("../snake-species-identification-challenge/data/round4_classes.csv")
#df1 = pd.read_csv("../snake-species-identification-challenge/data/train_labels.csv",dtype={'hashed_id':str})
#df2 = pd.read_csv("../snake-species-identification-challenge/data/validate_labels.csv",dtype={'hashed_id':str})
#df3 = pd.read_csv("../snake-species-identification-challenge/data/validate_labels_small.csv",dtype={'hashed_id':str})
#add_label(df, df1, df2, df3)

In [2]:
### Path settings
train_fraction = 1
val_fraction = 1
data_address = "../data/" # Change this to load files
train_label_filename = "processed_train_labels.csv"  # New Training 
val_label_filename = "processed_validate_labels.csv"
val_label_small_filename = "processed_validate_labels_small.csv"

### Load Training Set and Validation Set
training_dataset = SnakeDataSet(filename=os.path.join(data_address,train_label_filename),image_path=os.path.join(data_address,'resized_train_images_250')) 
val_dataset = SnakeDataSet(filename=os.path.join(data_address,val_label_filename),image_path=os.path.join(data_address,'resized_val_images_250')) 
training_size = len(training_dataset)
val_size = len(val_dataset)

# There are 783 classes (0-782)

### Load Set to Loader
train_loader = DataLoader(training_dataset, batch_size=64, sampler = SubsetRandomSampler(range(int(training_size / train_fraction))))
train_acc_loader = DataLoader(training_dataset, batch_size=64, sampler = SubsetRandomSampler(range(int(training_size / train_fraction))))
val_loader = DataLoader(val_dataset, batch_size=64, sampler = SubsetRandomSampler(range(int(val_size / val_fraction))))

### Clear running log
f = open('running.txt','w')
f.truncate(0)
f.close()

In [4]:
#filepath = '../snake-species-identification-challenge/data/resized_train_images/ffffaaf775.jpg'
#modelpath = '../snake-species-identification-challenge/alexnet_model_epoch30_lr1_batchsize64.pt'

#model = initialize_model('alexnet', 783, True, use_pretrained = True)
#model.load_state_dict(torch.load(modelpath))  
#device = torch.device("cuda")
#model = model.float()
#model = model.to(device)
#model.eval()
#for data0, target0 in val_loader:
#    data, target = data0.to(device), target0.to(device)
#    print(data.shape)
#    output = model(data.float()) 
#    print(np.shape(output))
#    break

In [3]:
# Random Seed
torch.manual_seed(2020)
np.random.seed(2020)

# Fine tuning parameters
pretrained = True
classifier_only = False

# Device
device = torch.device("cuda")
#model = BasicNet().to(device)          #2-layer CNN 
#model = AlexNet().to(device)           #alexnet
model = initialize_model('resnet', 783, classifier_only)
model = model.float()
model = model.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model.parameters()
print("Params to learn:")
if classifier_only:
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Optimizer
optimizer = optim.Adadelta(model.parameters(), lr=1)

# Set your learning rate scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

# Epochs
epochs = 10

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.b

In [4]:
### Training loop
train_loss = []
val_loss = []
for epoch in range(1, epochs+1,1):
    ts = time.time()
    train(model, device, train_loader, train_fraction, optimizer, epoch, 500)
    train_loss.append(test(model, device, train_acc_loader, train_fraction))
    val_loss.append(test(model, device, val_loader, val_fraction))
    scheduler.step()    # learning rate scheduler
    
    #Time an entire epoch
    te = time.time()
    message  = 'Time elapsed during the entire epoch: %.2f s.\n'%(te-ts)
    print(message)
    f = open('running.txt','a')
    f.write(message)
    f.close()

Train Epoch: 1 [0/245082 (0%)]	Loss: 6.825128

Time Elapsed: 1 s.

Train Epoch: 1 [32000/245082 (13%)]	Loss: 4.383014

Time Elapsed: 234 s.

Train Epoch: 1 [64000/245082 (26%)]	Loss: 3.785078

Time Elapsed: 235 s.

Train Epoch: 1 [96000/245082 (39%)]	Loss: 3.947802

Time Elapsed: 235 s.

Train Epoch: 1 [128000/245082 (52%)]	Loss: 3.301928

Time Elapsed: 235 s.

Train Epoch: 1 [160000/245082 (65%)]	Loss: 3.553214

Time Elapsed: 235 s.

Train Epoch: 1 [192000/245082 (78%)]	Loss: 3.315099

Time Elapsed: 235 s.

Train Epoch: 1 [224000/245082 (91%)]	Loss: 3.658424

Time Elapsed: 235 s.


Test set: Average loss: 42.9824, Accuracy: 62813/245082 (26%)


Test set: Average loss: 42.9388, Accuracy: 3524/14021 (25%)

Time elapsed during the entire epoch: 3455.73 s.

Train Epoch: 2 [0/245082 (0%)]	Loss: 3.407383

Time Elapsed: 0 s.

Train Epoch: 2 [32000/245082 (13%)]	Loss: 3.002714

Time Elapsed: 234 s.

Train Epoch: 2 [64000/245082 (26%)]	Loss: 3.017300

Time Elapsed: 234 s.

Train Epoch: 2 [9600

In [5]:
torch.save(model.state_dict(), "pretrained_resnet_model_epoch30_lr1_batchsize64.pt")
f = open("pretrained_resnet_model_epoch30_lr1_batchsize64_accuracy.txt","w")
f.write('Training Loss: ' + ' '.join(str(_) for _ in train_loss) + '\n')
f.write('Validation Loss: ' + ' '.join(str(_) for _ in val_loss) + '\n')
f.close()